In [ ]:
import pandas as pd
import requests
import json
import sys
import time

獲取隊伍資料

In [ ]:
r = requests.get(
    "https://bdfed.stitch.mlbinfra.com/bdfed/stats/team?stitch_env=prod&sportId=1&gameType=R&group=hitting&stats=season&season=2023&limit=30&offset=0",
)

In [ ]:
teams = json.loads(r.content)

In [ ]:
teams

In [ ]:
team_name_id= {team["teamShortName"]: team["teamId"] for team in teams["stats"]}

In [ ]:
team_name_id

獲取打者資料

In [ ]:
batters = list()

for key in team_name_id.keys():
    r = requests.get(
        f"https://bdfed.stitch.mlbinfra.com/bdfed/stats/player?stitch_env=prod&season=2023&sportId=1&stats=season&group=hitting&gameType=R&limit=50&offset=0&sortStat=homeRuns&order=desc&teamId={team_name_id[key]}"
    )
    players = json.loads(r.content)

    for player in players["stats"]:
        player_tuple = player["playerId"], player["playerName"], key
        batters.append(player_tuple)


batters[:10]

In [ ]:
len(batters)

### 主程式

生涯對決

In [ ]:
def pitcher_vs_batter(x,y):    
    i = 0
    j = x
    for batter in batters[x:y]:
        player_name = []
        player_team = []
        pitcher_name = []
        pitcher_team = []
        ab = []
        h = []
        twob = []
        threeb = []
        hr = []
        rbi = []
        bb = []
        ibb = []
        so = []
        avg = []
        obp = []
        slg = []
        ops = []
        playerId, playerName, teamName = batter
        for team in team_name_id.keys():
            if teamName == team:
                continue
            
            url = f"https://statsapi.mlb.com/api/v1/people/{playerId}/stats?stats=vsTeam&group=hitting&opposingTeamId={team_name_id[team]}&season=2023&language=en"
            #print(url)
            r = requests.get(url)
            player_json = json.loads(r.content)
            
            for data_type in player_json["stats"]:
                if data_type["type"]["displayName"] == "vsTeamTotal":
                    continue
                for split in data_type["splits"]:
                    split_stat = split["stat"]
                    player_name.append(split["batter"]["fullName"])
                    player_team.append(split["team"]["name"])
                    pitcher_name.append(split["pitcher"]["fullName"])
                    pitcher_team.append(split["opponent"]["name"])
                    ab.append(split_stat["atBats"])
                    h.append(split_stat["hits"])
                    twob.append(split_stat["doubles"])
                    threeb.append(split_stat["triples"])
                    hr.append(split_stat["homeRuns"])
                    rbi.append(split_stat["rbi"])
                    bb.append(split_stat["baseOnBalls"])
                    ibb.append(split_stat["intentionalWalks"])
                    so.append(split_stat["strikeOuts"])
                    avg.append(split_stat["avg"])
                    obp.append(split_stat["obp"])
                    slg.append(split_stat["slg"])
                    ops.append(split_stat["ops"])


        df = pd.DataFrame(
            {
                "player_name": player_name,
                "player_team": player_team,
                "pitcher_name": pitcher_name,
                "pitcher_team": pitcher_team,
                "ab": ab,
                "h": h,
                "2b": twob,
                "3b": threeb,
                "hr": hr,
                "rbi": rbi,
                "bb": bb,
                "ibb": ibb,
                "so": so,
                "avg": avg,
                "obp": obp,
                "slg": slg,
                "ops": ops,
            }
        )
        file_name = "pitcher_vs_batter/" + playerName + ".csv"
        df.to_csv(file_name)
        print(i+j)
        print("succssesfully export " + playerName + ".csv")
        i = i + 1

In [ ]:
x = 677
y = 678
pitcher_vs_batter(x, y)

打者近況

In [ ]:
def career(x,y):    
    i = 0
    j = x
    for batter in batters[x:y]:
        season = []
        ab = []
        h = []
        twob = []
        threeb = []
        hr = []
        rbi = []
        bb = []
        ibb = []
        so = []
        avg = []
        obp = []
        slg = []
        ops = []
        playerId, playerName, teamName = batter
        for team in team_name_id.keys():
            if teamName == team:
                continue
            url = f"https://statsapi.mlb.com/api/v1/people/{playerId}/stats?stats=yearByYear,career,yearByYearAdvanced,careerAdvanced&gameType=R&leagueListId=mlb_hist&group=hitting&hydrate=team(league)&language=en"
            #print(url)
            r = requests.get(url)
            player_json = json.loads(r.content)
            
            for data_type in player_json["stats"]:
                if data_type["type"]["displayName"] == "yearByYear":
                    for split in data_type["splits"]:
                        #print(split.keys())
                        season.append(split["season"])
                        split_stat = split["stat"]
                        ab.append(split_stat["atBats"])
                        h.append(split_stat["hits"])
                        twob.append(split_stat["doubles"])
                        threeb.append(split_stat["triples"])
                        hr.append(split_stat["homeRuns"])
                        rbi.append(split_stat["rbi"])
                        bb.append(split_stat["baseOnBalls"])
                        ibb.append(split_stat["intentionalWalks"])
                        so.append(split_stat["strikeOuts"])
                        avg.append(split_stat["avg"])
                        obp.append(split_stat["obp"])
                        slg.append(split_stat["slg"])
                        ops.append(split_stat["ops"])


        df = pd.DataFrame(
            {
                "Season":season,
                "ab": ab,
                "h": h,
                "2b": twob,
                "3b": threeb,
                "hr": hr,
                "rbi": rbi,
                "bb": bb,
                "ibb": ibb,
                "so": so,
                "avg": avg,
                "obp": obp,
                "slg": slg,
                "ops": ops,
            }
        )
        file_name = "career/" + playerName + ".csv"
        df.to_csv(file_name)
        print(i+j)
        print("succssesfully export " + playerName + ".csv")
        i = i + 1


In [ ]:
x = 677
y = 678
career(x, y)

https://ws.statsapi.mlb.com/api/v1.1/game/717505/feed/live?language=en

In [ ]:
def pinch_hitter(x,y):    
    i = 0
    j = x
    for batter in batters[x:y]:
        season = []
        ab = []
        h = []
        twob = []
        threeb = []
        hr = []
        rbi = []
        bb = []
        ibb = []
        so = []
        avg = []
        obp = []
        slg = []
        ops = []
        playerId, playerName, teamName = batter
        for team in team_name_id.keys():
            if teamName == team:
                continue
            url = f"https://statsapi.mlb.com/api/v1/people/{playerId}/stats?stats=yearByYear,career,yearByYearAdvanced,careerAdvanced&gameType=R&leagueListId=mlb_hist&group=hitting&hydrate=team(league)&language=en"
            #print(url)
            r = requests.get(url)
            player_json = json.loads(r.content)
            
            for data_type in player_json["stats"]:
                if data_type["type"]["displayName"] == "yearByYear":
                    for split in data_type["splits"]:
                        #print(split.keys())
                        season.append(split["season"])
                        split_stat = split["stat"]
                        ab.append(split_stat["atBats"])
                        h.append(split_stat["hits"])
                        twob.append(split_stat["doubles"])
                        threeb.append(split_stat["triples"])
                        hr.append(split_stat["homeRuns"])
                        rbi.append(split_stat["rbi"])
                        bb.append(split_stat["baseOnBalls"])
                        ibb.append(split_stat["intentionalWalks"])
                        so.append(split_stat["strikeOuts"])
                        avg.append(split_stat["avg"])
                        obp.append(split_stat["obp"])
                        slg.append(split_stat["slg"])
                        ops.append(split_stat["ops"])


        df = pd.DataFrame(
            {
                "Season":season,
                "ab": ab,
                "h": h,
                "2b": twob,
                "3b": threeb,
                "hr": hr,
                "rbi": rbi,
                "bb": bb,
                "ibb": ibb,
                "so": so,
                "avg": avg,
                "obp": obp,
                "slg": slg,
                "ops": ops,
            }
        )
        file_name = "career/" + playerName + ".csv"
        df.to_csv(file_name)
        print(i+j)
        print("succssesfully export " + playerName + ".csv")
        i = i + 1
